In [17]:
import os
from shutil import copyfile
import itertools

In [18]:
basedir = "Experiments/TXF_0.01/logs"
experiment_basedir = "Experiments/TXF_0.01/analysis"

In [19]:
experiments = os.listdir(basedir)

In [20]:
variables = ["mean_usd", "stddev_usd", "mean_dai", "stddev_dai"]

In [21]:
filter_var = variables[3]

In [22]:
def get_filter_vars_from_path(path, filter_vars):
    path_split = path[12:-1].split(',')
        
    var_dict = {}
    
    for i in path_split:
        key, val = i.split(':')
        var_dict[key] = val 

    return [var_dict[filter_var] for filter_var in filter_vars]

def get_directory_name(filter_directory, filter_vars, filter_var_vals):
    directory_name = ""
    for k in range(len(filter_var_vals)):
        directory_name += (filter_vars[k] + ":" + filter_var_vals[k] + " ")

    directory_name = directory_name[:-1]
    
    return os.path.join(experiment_basedir, filter_directory, directory_name)
    

def filter_results(filter_vars):
    filter_directory = "filter:" + ":".join(filter_vars)
        
    # get possible variable values    
    filter_val_list = []
    for filter_var in filter_vars:
        filter_val_list.append(list(set(sorted([get_filter_vars_from_path(i, [filter_var])[0] for i in os.listdir(basedir)]))))    
    
    # construct temp directories for each var combination
    for i in itertools.product(*filter_val_list):
        os.makedirs(get_directory_name(filter_directory, filter_vars, i), exist_ok = True)
        
    # copy plots to respective directories
    for i in os.listdir(basedir):
        filter_var_vals = get_filter_vars_from_path(i, filter_vars)
        
        source = os.path.join(basedir, i, "plots", "final_settling_price_txf_on_x_0.png")
        destination = os.path.join(get_directory_name(filter_directory, filter_vars, filter_var_vals), i + ".png")
        #print(source, destination)
        copyfile(source, destination)

### Filter Plots based on a variable list

In [23]:
for i in range(1, 15):
    var_list = []
    for k in range(4):
        if format(i, '04b')[k] == '1':
            var_list.append(variables[k])
    filter_results(var_list)

### Filter plots based on risk and percentage of risky investors

In [24]:
investor_count = 10
asset_curve_plot_name = "asset_curve_cdp_rate_0.02_run_0.jpeg"

In [25]:
def get_risk_param_from_path(path):
    risk_bitmask = get_filter_vars_from_path(path, ["risk_bitmask"])[0]
    return list(risk_bitmask)
    
def is_bit_set(risk_bitmask, bit_number):
    return risk_bitmask[bit_number] == '1'

def percentage_unset_bits(risk_bitmask):
    return round((risk_bitmask.count('0') / len(risk_bitmask)) * 100, 2)

def filter_results_by_risk(num_investors):
    filter_directory = "filter:risk"
    
    # generate risk directories
    for i in range(num_investors):
        os.makedirs(os.path.join(experiment_basedir, filter_directory, "risk:investor_" + str(i) + "_high"), exist_ok=True)
        os.makedirs(os.path.join(experiment_basedir, filter_directory, "risk:investor_" + str(i) + "_low"), exist_ok = True)
    
    for perc in [round((100 * i) / num_investors, 2) for i in range(0, num_investors + 1)]:
        os.makedirs(os.path.join(experiment_basedir, filter_directory, "risk:" + str(perc) + "_perc_hi_risk"), exist_ok = True)
        
    # copy plots to respective directories
    for i in os.listdir(basedir):
        risk_bitmask = get_risk_param_from_path(i)    
        source = os.path.join(basedir, i, "plots", asset_curve_plot_name)
        
        # copy to required precentage directory
        perc_hi_risk = percentage_unset_bits(risk_bitmask)
        perc_hi_risk_destination = os.path.join(experiment_basedir, filter_directory, "risk:" + str(perc_hi_risk) + "_perc_hi_risk", i + ".jpeg")
        
        copyfile(source, perc_hi_risk_destination)    
            
        for inv in range(num_investors):
            is_set = is_bit_set(risk_bitmask, inv)
            
            if is_set:
                    destination = os.path.join(experiment_basedir, filter_directory, "risk:investor_" + str(inv) + "_high", i + ".jpeg")
                    copyfile(source, destination)
        
        
        for inv in range(num_investors):
            is_set = is_bit_set(risk_bitmask, inv)
            
            if not is_set:
                    destination = os.path.join(experiment_basedir, filter_directory, "risk:investor_" + str(inv) + "_low", i + ".jpeg")
                    copyfile(source, destination)

In [26]:
filter_results_by_risk(investor_count)

### Filter plots based on percentage of risky investors